In [ ]:
# Wait 60 seconds before connecting using these details, or login to https://console.neo4j.io to validate the Aura Instance is available
%%writefile .env
NEO4J_URI='neo4j+s://7db34ce8.databases.neo4j.io'
NEO4J_USERNAME='neo4j'
NEO4J_PASSWORD= 'MR-7RyVtPsKvsHyxb3Z2ZRGBbR7M7jBOtu4h3Q-GrIg'
AURA_INSTANCEID='7db34ce8'
AURA_INSTANCENAME='Instance01'

Writing .env


In [ ]:
!pip install -q python-dotenv

In [ ]:
from dotenv import load_dotenv
import os

# Common data processing
import textwrap

# Langchain
# from langchain_community.graphs import Neo4jGraph
# from langchain_community.vectorstores import Neo4jVector
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.chains import RetrievalQAWithSourcesChain
# from langchain.llms import OpenAI
# from langchain.embeddings import OpenAIEmbeddings
# from langchain.document_loaders import PyPDFLoader

In [ ]:
!pip install -q langchain

In [ ]:
!pip install -q langchain-community

In [ ]:
from langchain_community.graphs import Neo4jGraph

In [ ]:
from langchain_community.vectorstores import Neo4jVector

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain

In [ ]:
from langchain.llms import OpenAI

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
from langchain.document_loaders import PyPDFLoader

In [ ]:
!pip install -q pypdf

In [ ]:
loader = PyPDFLoader("/content/drive/MyDrive/mystic/vachnamrut.pdf")
pages = loader.load_and_split()

In [ ]:
print(len(pages))

In [ ]:
print(pages[0])

In [ ]:
print(pages)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks = text_splitter.split_documents(pages)


In [ ]:
print(len(chunks))

In [ ]:
print(chunks[2263])

In [ ]:
!pip install -q neo4j

In [ ]:
# Warning control
import warnings
warnings.filterwarnings("ignore")

# Load from environment from the credentials file
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')

NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')

NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'
NEO4J_DATABASE = 'neo4j'
# Global constants
VECTOR_INDEX_NAME = 'pdf_chunks'
VECTOR_NODE_LABEL = 'Chunk'
VECTOR_SOURCE_PROPERTY = 'text'
VECTOR_EMBEDDING_PROPERTY = 'textEmbedding'

kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [ ]:
%pip install --upgrade --quiet  langchain-google-genai

In [ ]:
import getpass
import os

os.environ["GOOGLE_API_KEY"] = getpass.getpass()

··········


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [ ]:
# Create Neo4j vector store
neo4j_vector_store = Neo4jVector.from_documents(
    embedding=GoogleGenerativeAIEmbeddings(model='models/embedding-001'),
    documents=chunks,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name=VECTOR_INDEX_NAME,
    text_node_property=VECTOR_SOURCE_PROPERTY,
    embedding_node_property=VECTOR_EMBEDDING_PROPERTY,
)

In [ ]:
# Create a PDF node
cypher = """
MERGE (p:PDF {name: $pdfName})
RETURN p
"""
kg.query(cypher, params={'pdfName': "/content/drive/MyDrive/mystic/vachnamrut.pdf"})

# Connect chunks to their parent PDF with a PART_OF relationship
cypher = """
MATCH (c:Chunk), (p:PDF)
WHERE p.name = $pdfName
MERGE (c)-[newRelationship:PART_OF]->(p)
RETURN count(newRelationship)
"""
kg.query(cypher, params={'pdfName': "/content/drive/MyDrive/mystic/vachnamrut.pdf"})

# Create a NEXT relationship between subsequent chunks
cypher = """
MATCH (c1:Chunk {pdfName: $pdfName}), (c2:Chunk {pdfName: $pdfName})
WHERE c1.chunkSeqId = c2.chunkSeqId - 1
MERGE (c1)-[r:NEXT]->(c2)
RETURN count(r)
"""
kg.query(cypher,params={'pdfName': "/content/drive/MyDrive/mystic/vachnamrut.pdf"})

[{'count(r)': 0}]

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
!pip install -qU langchain-google-vertexai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 kB 7.7 MB/s eta 0:00:00


In [ ]:
from langchain_google_vertexai import ChatVertexAI

In [ ]:
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
os.environ["LANGSMITH_TRACING"] = "true"

Enter your LangSmith API key: ··········


In [ ]:
os.environ["GOOGLE_CLOUD_PROJECT"] = "banded-splicer-427209-q9"

In [ ]:
# Create a retriever from the vector store
retriever = neo4j_vector_store.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatGoogleGenerativeAI(temperature=0, model="gemini-1.5-pro"),
    chain_type="stuff",
    retriever=retriever
)

# Ask a question
question = " give the title of vachanamrut gadhada pratham first"
answer = chain(
    {"question": question},
    return_only_outputs=True,
)
print(textwrap.fill(answer["answer"]))

In [ ]:
# Returns the node count
kg.query("""
         MATCH (n)
         RETURN count(n) as nodeCount
         """)

[{'nodeCount': 6285}]

In [ ]:
# kg.refresh_schema()
# print(kg.schema)

In [ ]:
# kg.query("SHOW INDEXES")

[{'id': 3,
  'name': 'constraint_1dc138a',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'RANGE',
  'entityType': 'NODE',
  'labelsOrTypes': ['Chunk'],
  'properties': ['id'],
  'indexProvider': 'range-1.0',
  'owningConstraint': 'constraint_1dc138a',
  'lastRead': neo4j.time.DateTime(2024, 6, 21, 18, 14, 54, 16000000, tzinfo=<UTC>),
  'readCount': 6788},
 {'id': 0,
  'name': 'index_343aff4e',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'NODE',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 6, 21, 18, 20, 27, 637000000, tzinfo=<UTC>),
  'readCount': 70},
 {'id': 1,
  'name': 'index_f7700477',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'RELATIONSHIP',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': None